In [1]:
"""
Uses cosine similarity.
Filters only books where at least 100 reviews are present.
Only where reviewers have given at least 200 reviews.
"""


import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split



In [2]:
books_df = pd.read_csv("../data/Books.csv")
users_df = pd.read_csv("../data/Users.csv")
ratings_df = pd.read_csv("../data/ratings.csv")

C:\Users\NelsonWang\AppData\Local\Temp\ipykernel_11748\3971019450.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv("../data/Books.csv")


In [3]:
ratings_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [4]:
# Clean up
# 1. Find the user ID of those who have rater either too few books (<5) and too many books (>200)
# 2. Remove those users from the ratings_df

valid_transaction_df = ratings_df.groupby('User-ID').filter(lambda x: len(x) > 5 and len(x) < 200)
valid_transaction_df.groupby('User-ID').size().sort_values(ascending=False)

User-ID
240403    199
203017    199
193458    199
2033      198
267061    198
         ... 
95420       6
95156       6
231313      6
95146       6
233397      6
Length: 18812, dtype: int64

In [5]:
ratings_with_book_titles = ratings_df.merge(books_df,on='ISBN')
ratings_with_book_titles.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,276726,0155061224,5,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...
2,276727,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
3,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...
4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...


In [6]:
ratings_with_book_titles.drop(columns=["ISBN","Image-URL-S","Image-URL-M"],axis=1,inplace=True)

In [7]:
complete_df = ratings_with_book_titles.merge(users_df.drop("Age", axis=1), on="User-ID")
complete_df.head()

,User-ID,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,Location
0,276725,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,"tyler, texas, usa"
1,276726,5,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...,"seattle, washington, usa"
2,276727,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,"h, new south wales, australia"
3,276729,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,http://images.amazon.com/images/P/052165615X.0...,"rijeka, n/a, croatia"
4,276729,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,http://images.amazon.com/images/P/0521795028.0...,"rijeka, n/a, croatia"


In [8]:
complete_df['Location'] = complete_df['Location'].str.split(',').str[-1].str.strip()
complete_df.head()

,User-ID,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,Location
0,276725,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,usa
1,276726,5,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...,usa
2,276727,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,australia
3,276729,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,http://images.amazon.com/images/P/052165615X.0...,croatia
4,276729,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,http://images.amazon.com/images/P/0521795028.0...,croatia


In [9]:
min_user_reviewers = 200
user_ratings = complete_df.groupby('User-ID').count()
filtered_users = user_ratings[user_ratings > min_user_reviewers].index
complete_df = complete_df[complete_df['User-ID'].isin(filtered_users)]


In [10]:
min_ratings_count_threshold=100
rating_counts= complete_df.groupby('Book-Title').count()['Book-Rating']
popular_books = rating_counts[rating_counts >= min_ratings_count_threshold].index

In [11]:
final_ratings =  complete_df[complete_df['Book-Title'].isin(popular_books)]
print(f"Number of ratings: {len(final_ratings)}")
final_ratings.head()

Number of ratings: 183799


,User-ID,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,Location
2,276727,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,australia
6,276744,7,A Painted House,JOHN GRISHAM,2001,Doubleday,http://images.amazon.com/images/P/038550120X.0...,usa
7,276746,0,Lightning,Dean R. Koontz,1996,Berkley Publishing Group,http://images.amazon.com/images/P/0425115801.0...,
8,276746,0,Manhattan Hunt Club,JOHN SAUL,2002,Ballantine Books,http://images.amazon.com/images/P/0449006522.0...,
9,276746,0,Dark Paradise,TAMI HOAG,1994,Bantam,http://images.amazon.com/images/P/0553561618.0...,


In [12]:
pt = final_ratings.pivot_table(index='Book-Title',columns='User-ID'
                          ,values='Book-Rating')
pt.head()

User-ID,8,9,14,16,17,26,32,39,42,44,...,278819,278820,278824,278828,278832,278836,278843,278844,278846,278854
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24 Hours,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
pt.fillna(0,inplace=True)
pt

User-ID,8,9,14,16,17,26,32,39,42,44,...,278819,278820,278824,278828,278832,278836,278843,278844,278846,278854
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24 Hours,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
from sklearn.metrics.pairwise import cosine_similarity 

similarity_score = cosine_similarity(pt)


def recommend(book_name):
    index = np.where(pt.index==book_name)[0][0]
    similar_books = sorted(list(enumerate(similarity_score[index])),key=lambda x:x[1], reverse=True)[1:6]
    
    data = []
    
    for i in similar_books:
        item = []
        temp_df = books_df[books_df['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        
        data.append(item)
    return data

def print_available_books():
    return list(final_ratings['Book-Title'].unique())

def search_name_like(name):
    return list(final_ratings[final_ratings['Book-Title'].str.contains(name)]['Book-Title'].unique())

def search_and_recommend(name):
    search_results = search_name_like(name)
    if len(search_results) == 0:
        return "No books found"
    return search_results, recommend(search_results[0])

In [15]:
search_name_like("Heaven")

['Heaven and Earth (Three Sisters Island Trilogy)',
 'One Door Away from Heaven',
 'The Five People You Meet in Heaven',
 'Pigs in Heaven',
 'Cry to Heaven']

In [16]:
recommend("One Door Away from Heaven")

[['From a Buick 8 : A Novel',
  'Stephen King',
  'http://images.amazon.com/images/P/0743211375.01.MZZZZZZZ.jpg'],
 ['By the Light of the Moon',
  'DEAN KOONTZ',
  'http://images.amazon.com/images/P/0553582763.01.MZZZZZZZ.jpg'],
 ['Mr. Murder',
  'Dean R. Koontz',
  'http://images.amazon.com/images/P/0425144429.01.MZZZZZZZ.jpg'],
 ['The Key to Midnight',
  'Dean R. Koontz',
  'http://images.amazon.com/images/P/0425147517.01.MZZZZZZZ.jpg'],
 ['Dark Rivers of the Heart',
  'Dean R. Koontz',
  'http://images.amazon.com/images/P/034539657X.01.MZZZZZZZ.jpg']]

In [17]:
recommend('Harry Potter and the Chamber of Secrets (Book 2)')

[['Harry Potter and the Prisoner of Azkaban (Book 3)',
  'J. K. Rowling',
  'http://images.amazon.com/images/P/0439136350.01.MZZZZZZZ.jpg'],
 ['Harry Potter and the Goblet of Fire (Book 4)',
  'J. K. Rowling',
  'http://images.amazon.com/images/P/0439139597.01.MZZZZZZZ.jpg'],
 ["Harry Potter and the Sorcerer's Stone (Book 1)",
  'J. K. Rowling',
  'http://images.amazon.com/images/P/0590353403.01.MZZZZZZZ.jpg'],
 ["Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))",
  'J. K. Rowling',
  'http://images.amazon.com/images/P/059035342X.01.MZZZZZZZ.jpg'],
 ['Harry Potter and the Order of the Phoenix (Book 5)',
  'J. K. Rowling',
  'http://images.amazon.com/images/P/043935806X.01.MZZZZZZZ.jpg']]

In [18]:
search_and_recommend('Dune')

(['Dune (Remembering Tomorrow)'],
 [['The Mists of Avalon',
   'MARION ZIMMER BRADLEY',
   'http://images.amazon.com/images/P/0345350499.01.MZZZZZZZ.jpg'],
  ['Silence of the Lambs',
   'Thomas Harris',
   'http://images.amazon.com/images/P/0312924585.01.MZZZZZZZ.jpg'],
  ['Skeleton Crew',
   'Stephen King',
   'http://images.amazon.com/images/P/0451168615.01.MZZZZZZZ.jpg'],
  ["Ender's Game (Ender Wiggins Saga (Paperback))",
   'Orson Scott Card',
   'http://images.amazon.com/images/P/0312853238.01.MZZZZZZZ.jpg'],
  ["Dirk Gently's Holistic Detective Agency",
   'Douglas Adams',
   'http://images.amazon.com/images/P/0671660632.01.MZZZZZZZ.jpg']])

In [19]:
search_and_recommend('Mystery')

(['E Is for Evidence: A Kinsey Millhone Mystery (Kinsey Millhone Mysteries (Paperback))'],
 [['D Is for Deadbeat (Kinsey Millhone Mysteries (Paperback))',
   'Sue Grafton',
   'http://images.amazon.com/images/P/0553271636.01.MZZZZZZZ.jpg'],
  ['F Is for Fugitive (Kinsey Millhone Mysteries (Paperback))',
   'Sue Grafton',
   'http://images.amazon.com/images/P/0553284789.01.MZZZZZZZ.jpg'],
  ['I Is for Innocent',
   'Sue Grafton',
   'http://images.amazon.com/images/P/0449221512.01.MZZZZZZZ.jpg'],
  ['H Is for Homicide (Kinsey Millhone Mysteries (Paperback))',
   'Sue Grafton',
   'http://images.amazon.com/images/P/0449219461.01.MZZZZZZZ.jpg'],
  ['G Is for Gumshoe (Kinsey Millhone Mysteries (Paperback))',
   'Sue Grafton',
   'http://images.amazon.com/images/P/0449219364.01.MZZZZZZZ.jpg']])

In [20]:
search_and_recommend('American Gods')

(['American Gods'],
 [['Neverwhere',
   'Neil Gaiman',
   'http://images.amazon.com/images/P/0380789019.01.MZZZZZZZ.jpg'],
  ['Coraline',
   'Neil Gaiman',
   'http://images.amazon.com/images/P/0380977788.01.MZZZZZZZ.jpg'],
  ['Stardust',
   'Neil Gaiman',
   'http://images.amazon.com/images/P/156389470X.01.MZZZZZZZ.jpg'],
  ['The Amber Spyglass (His Dark Materials, Book 3)',
   'PHILIP PULLMAN',
   'http://images.amazon.com/images/P/0375823352.01.MZZZZZZZ.jpg'],
  ['Good Omens',
   'Neil Gaiman',
   'http://images.amazon.com/images/P/0441003257.01.MZZZZZZZ.jpg']])